In [6]:
from keras.utils import np_utils
import numpy as np
import tensorflow as tf
import os
from PIL import Image
from PIL import ImageFilter
import image_utils as iu
import uuid
from shutil import copy2

In [7]:
data_path = 'data'
object_detection_path = os.path.join(data_path, 'object-detection')
# segmentation_path = os.path.join(data_path, 'segmentation')
#Mining 1 contains satellite images from Google Earth taken with screenshots.
# mining_path = os.path.join(segmentation_path, 'mining')
# divided images
object_detection_original_data_path = os.path.join(object_detection_path, 'object-detection-original')
# mining_real_divided_data_path = os.path.join(mining_divided_data_path, 'real')
# mining_mask_divided_data_path = os.path.join(mining_divided_data_path, 'mask')
# classification images
object_detection_divided_data_path = os.path.join(
    object_detection_path, 'object-detection-divided')
# mining_site_divided_path = os.path.join(
#     classification_divided_data_path, 'mining-site')
# others_divided_path = os.path.join(classification_divided_data_path, 'others')
# #
# classification_final_data_path = os.path.join(
#     classification_path, 'classification-final')
# mining_site_final_path = os.path.join(
#     classification_final_data_path,  'mining-site')
# others_final_data_path = os.path.join(classification_final_data_path, 'others')
#train
# training_path = os.path.join(classification_path, 'training')
# train_path = os.path.join(training_path, 'train')
# train_mining_site_path = os.path.join(train_path, 'mining-site')
# train_others_path = os.path.join(train_path, 'others')
# #val
# val_path = os.path.join(training_path, 'val')
# val_mining_site_path = os.path.join(val_path, 'mining-site')
# val_others_path = os.path.join(val_path, 'others')


In [12]:
charcoal_images_path = iu.get_all_files(
    object_detection_original_data_path, True)


In [9]:
# Splits image into specified rows and columns
def split_image(image, row_count, col_count):
    parts = []
    width, height = image.size
    left = 0
    top = 0
    right = width / col_count
    bottom = height / row_count
    for r in range(row_count):
        top = int(r * (height / row_count))
        bottom = int(top + (height / row_count))
        for c in range(col_count):
            left = int(c * (width / col_count))
            right = int(left + (width / col_count))
            part = image.crop((left, top, right, bottom))
            parts.append(part)
    return parts

In [14]:
# images are divided into 9 parts and saved on disk.
# This reduces the dimension of the images and also increases the training data.
iu.create_dir_if_not_exists(object_detection_divided_data_path)
for charcoal_image in charcoal_images_path:
    row_count = 25
    col_count = 12
    row_count = 8
    col_count = 4
    row_count = 8
    col_count = 12
    part_count = col_count * row_count
    real_image = iu.read_image(charcoal_image, 'RGB')
    real_image_parts = split_image(real_image, row_count, col_count)
    count = 0
    for x in range(part_count):
        # part_filename = str(uuid.uuid4()) + '.jpg'
        part_filename = str(count) + '.jpg'
        real_image_part = real_image_parts[x]
        real_image_part.save(os.path.join(
            object_detection_divided_data_path, part_filename))
        count = count + 1
